At the end of this step, you will understand the concepts of underfitting and overfitting, and you will be able to apply these ideas to make your models more accurate.

bir sürü karar ağacı çeşidi vardır. en önemlileri deriniği belirleyenlerdir. bir ağacın derinliği, bir tahmine ulaşmadan önce kaç bölme yaptığına dair bir ölçüdür.
bir ağaç her bölündüğünde daha çok leaf ve her birinin içinde daha az ev bulunur. örneğin 10 split uygulanan bir ağaçta 2^10 = 1024 leaf meydana gelir.

Evleri birçok yaprak arasında böldüğümüzde, her yaprakta daha az sayıda ev kalır. 
Az sayıda ev içeren yapraklar, bu evlerin gerçek değerlerine çok yakın tahminler yapabilir, 
ancak yeni veriler için bu tahminler oldukça güvensiz olabilir (çünkü her tahmin yalnızca birkaç eve dayanır).

kendi eğitildiği verilerle harika çalışan ama başka verilerle çok kötü çalışan modellerin oluşturduğu duruma overfitting(aşırı uyum) denir.
Fakat diğer yandan ağacımızı az split edersek sığ(shallow) bir ağaç oluşur ve yeterince gruplama yapılamaz.

sığ bir ağaç oluşturulursa her bir leaf te çok fazla ev bulunur ve oluşan model hem train data ile hem de validation data ile kötü tahminler yapar.
Bir model, verideki önemli ayrımları ve desenleri yakalayamadığında ve bu yüzden eğitim verilerinde bile kötü performans gösterdiğinde, 
buna underfitting (aşırı basitleştirme) denir.

![Alt Metin](image.png)

Sweet Spot (Tatlı Nokta): Grafikte sarı bir uyarı işareti ile gösterilen orta bölge, modelin ne underfitting ne de overfitting yaptığı noktadır. Burada doğrulama hatası en düşük seviyededir. Model bu noktada genel olarak en iyi performansı gösterir.

Bu grafikteki temel amaç, modelin doğrulama verilerinde en düşük hatayı yaptığı "sweet spot"u bulmaktır. Bu, modelin verideki desenleri yeterince iyi yakaladığı ve yeni verilere genelleme yapabildiği noktadır.

Ağaç derinliğini kontrol etmek için birkaç alternatif vardır ve bunların çoğu, ağacın bazı dallarının diğerlerinden daha derin olmasına izin verir. Ancak, max_leaf_nodes argümanı, aşırı uyum (overfitting) ve yetersiz uyum (underfitting) arasında denge kurmanın çok mantıklı bir yolunu sunar. Modele daha fazla yaprak oluşturmasına izin verdikçe, yukarıdaki grafikte yetersiz uyum bölgesinden aşırı uyum bölgesine doğru ilerleriz.

max_leaf_nodes için farklı değerler kullanarak MAE (Ortalama Mutlak Hata) puanlarını karşılaştırmaya yardımcı olmak için bir yardımcı işlev kullanabiliriz.

In [16]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

import pandas as pd
from sklearn.model_selection import train_test_split

In [17]:
def getmae(max_leaf_nodes, train_X, train_y, val_X, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return mae

In [18]:
melbourneData = "C:/Users/Alperen Arda/OneDrive/Desktop/GitHub/Data-Science-and-Machine-Learning-Tutorial/Datasets/melb_data.csv"
houses_data = pd.read_csv(melbourneData)

filtered_melbourne_data = houses_data.dropna(axis=0)

y = filtered_melbourne_data.Price

melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']

X = filtered_melbourne_data[melbourne_features]

train_X, train_y, val_X, val_y = train_test_split(X, y, random_state=0)

In [24]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = getmae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5 		 Mean Absolute Error:  347380
Max leaf nodes: 50 		 Mean Absolute Error:  258171
Max leaf nodes: 500 		 Mean Absolute Error:  246793
Max leaf nodes: 5000 		 Mean Absolute Error:  261451


Of the options listed, 500 is the optimal number of leaves.

In [ ]:
final_model = DecisionTreeRegressor(max_leaf_nodes=500, random_state=1)

final_model.fit(X, y)

Neden train_X ve train_y Kullanılmıyor?

Modelin Son Şekli: best_tree_size'ı belirledikten sonra, modelinizi optimize etmek için tüm veri setini kullanmanız gerekir. Bu, modelin eğitimi için en iyi sonucu sağlayabilir.

Doğrulama Verisi Ayrımı: Modelin eğitiminde artık doğrulama verisini kullanmanıza gerek kalmaz çünkü doğrulama süreci tamamlanmıştır. Bu nedenle, modelinizi tüm veri seti üzerinde eğitmek mantıklıdır.

Sonuç olarak, best_tree_size'ı belirledikten sonra, modeli en iyi performans için tüm verilerle eğitmek iyi bir uygulamadır.

_İşte özet_:

Modeller şu sorunlarla karşılaşabilir:

Aşırı Uydurma (Overfitting): Geçici desenleri yakalayıp gelecekte tekrar etmeyecek olanları öğrenir, bu da daha az doğru tahminler yapmasına yol açar.

Yetersiz Uydurma (Underfitting): İlgili desenleri yakalayamaz, bu da yine daha az doğru tahminler yapmasına yol açar.
Model eğitimi sırasında kullanılmayan validation dataları kullanarak candidate modelin doğruluğunu ölçeriz. Bu, birçok aday modeli denememize ve en iyisini seçmemize olanak tanır.

In [23]:
val_mae = 6482942.527
print("Validation MAE: {:,.0f}".format(val_mae))
# Burada , sayıları binlik ayrıcı (thousands separator) ile ayırır.

Validation MAE: 6,482,943


En iyi ağaç boyutunu biliyorsunuz. Bu modeli pratikte kullanacak olsanız, tüm verileri kullanarak ve bu ağaç boyutunu koruyarak modeli daha da doğru hale getirirsiniz. Yani, tüm modelleme kararlarınızı verdikten sonra doğrulama verisini artık saklamanıza gerek yok.

~HALA ANLAMADIYSAN~

max_leaf_nodes parametresi, karar ağaçlarında kullanılan bir hiperparametredir ve ağacın yapısını kontrol etmek için kullanılır. Şimdi açıklamaya çalışayım:

max_leaf_nodes Nedir?
max_leaf_nodes: Karar ağacındaki maksimum yaprak (leaf) sayısını sınırlar. Bir yaprak, ağacın son düğümüdür ve modelin nihai tahminlerini yapar. Bu parametre, ağacın toplamda kaç yaprak içerebileceğini belirler.
Nasıl Çalışır?
Daha Küçük max_leaf_nodes: Eğer max_leaf_nodes değeri küçükse, ağaç daha basit olur çünkü daha az yaprağa sahiptir. Bu durum:

Yetersiz Uydurma (Underfitting): Model, verilerdeki karmaşık desenleri öğrenemez ve dolayısıyla düşük doğruluk sağlar. Veriyi yeterince iyi modellemediği için eğitim verisine uyum sağlamaz.
Daha Büyük max_leaf_nodes: Eğer max_leaf_nodes değeri büyükse, ağaç daha karmaşık hale gelir ve daha fazla yaprak içerir. Bu durum:

Aşırı Uydurma (Overfitting): Model, eğitim verisindeki küçük varyasyonları öğrenir ve veriye fazla uyum sağlar. Bu, test verileri veya yeni veriler üzerinde kötü performansa yol açabilir çünkü model eğitim verisindeki gürültüyü öğrenmiş olabilir.
Denge Kurma
Denge Kurma: max_leaf_nodes değerini ayarlayarak, modelin karmaşıklığını kontrol edebilir ve aşırı uyum ile yetersiz uyum arasında bir denge kurabilirsiniz.
Eğer max_leaf_nodes çok düşükse, model yetersiz uyum sağlar.
Eğer max_leaf_nodes çok yüksekse, model aşırı uyum sağlar.
Grafik ve Uygulama
Grafikteki Denge: Grafik üzerinde, max_leaf_nodes değerini arttırdığınızda, modelin yetersiz uyum bölgesinden aşırı uyum bölgesine geçiş yaptığını görebilirsiniz. Bu, modelin başlangıçta daha basit ve genelleştirilmiş olduğunu, ancak daha karmaşık hale geldikçe eğitim verisine daha fazla uyum sağladığını gösterir.
Kısacası, max_leaf_nodes parametresi, ağacın ne kadar karmaşık olabileceğini kontrol eder ve bu, modelin performansını ve genelleme yeteneğini etkiler.